# Projet Final Apache Spark

**Nom Etudiant : DRAME**  

**Prenom Etudiant: NDEYE AIDA**  

**Classe : BIG DATA**  


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Samedi 31 Juillet 2021  à 23h 59** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-EKANUDC:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1628017285455)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._
import org.apache.spark.sql.SparkSession


In [6]:
val spark = SparkSession
    .builder()
    .config("spark.debug.maxToStringFields", "100")
    .appName("Data Processing")
    .master("local[8]")
    .getOrCreate

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1aea6394


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [7]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

head : Le terme head n'est pas reconnu comme nom d'applet de commande, fonction, fichier de script ou programme 
excutable. Vrifiez l'orthographe du nom, ou si un chemin d'accs existe, vrifiez que le chemin d'accs est correct 
et ressayez.
Au caractre Ligne:1 : 1
+ head -1 "datasets/sf-fire/sf-fire-calls.csv"
+ ~~~~
    + CategoryInfo          : ObjectNotFound: (head:String) [], CommandNotFoundException
    + FullyQualifiedErrorId : CommandNotFoundException
 



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [8]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [9]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [10]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [11]:
fireDF.count()

res1: Long = 175296


In [12]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [13]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [14]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [15]:
// Reponse 1
val typA = fewFireDF.select($"CallType").distinct().count()

typA: Long = 29


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [16]:
// Reponse 2
val typADist = fewFireDF.
             select($"CallType").
             distinct().
             show(29)

+--------------------+
|            CallType|
+--------------------+
|Elevator / Escala...|
|         Marine Fire|
|  Aircraft Emergency|
|Confined Space / ...|
|      Administrative|
|              Alarms|
|Odor (Strange / U...|
|Citizen Assist / ...|
|              HazMat|
|Watercraft in Dis...|
|           Explosion|
|           Oil Spill|
|        Vehicle Fire|
|  Suspicious Package|
|Extrication / Ent...|
|               Other|
|        Outside Fire|
|   Traffic Collision|
|       Assist Police|
|Gas Leak (Natural...|
|        Water Rescue|
|   Electrical Hazard|
|   High Angle Rescue|
|      Structure Fire|
|Industrial Accidents|
|Mutual Aid / Assi...|
|          Fuel Spill|
|Smoke Investigati...|
|Train / Rail Inci...|
+--------------------+



typADist: Unit = ()


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [17]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
val typA = newFireDF
         .select("ResponseDelayedinMins", "AvailableDtTm","CallType","Neighborhood")
         .filter($"ResponseDelayedinMins" > 5)
         .show(10)

+---------------------+--------------------+--------------------+--------------------+
|ResponseDelayedinMins|       AvailableDtTm|            CallType|        Neighborhood|
+---------------------+--------------------+--------------------+--------------------+
|                 5.35|01/11/2002 04:34:...|    Medical Incident|  West of Twin Peaks|
|                 6.25|01/12/2002 01:23:...|    Medical Incident|        Russian Hill|
|                  5.2|01/13/2002 01:51:...|    Medical Incident|          Tenderloin|
|                  5.6|01/14/2002 08:16:...|Citizen Assist / ...|     Sunset/Parkside|
|                 7.25|01/14/2002 12:27:...|    Medical Incident|Financial Distric...|
|            11.916667|01/15/2002 06:53:...|    Medical Incident|     South of Market|
|             5.116667|01/15/2002 03:18:...|    Medical Incident| Castro/Upper Market|
|             8.633333|01/15/2002 03:55:...|    Medical Incident|         North Beach|
|             95.28333|01/15/2002 07:49:...

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
typA: Unit = ()


### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [18]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res5: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [19]:
//Reponse 4
val appelCour = fireTSDF.select("CallType").
                  groupBy("CallType").count().
                  orderBy(desc("count"))
appelCour.show(10)

+--------------------+------+
|            CallType| count|
+--------------------+------+
|    Medical Incident|113794|
|      Structure Fire| 23319|
|              Alarms| 19406|
|   Traffic Collision|  7013|
|Citizen Assist / ...|  2524|
|               Other|  2166|
|        Outside Fire|  2094|
|        Vehicle Fire|   854|
|Gas Leak (Natural...|   764|
|        Water Rescue|   755|
+--------------------+------+
only showing top 10 rows



appelCour: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, count: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [20]:
//Reponse 5-a
val appelCour = fireTSDF.
                  groupBy($"CallType",$"Box").count().
                  orderBy(desc("count"))
appelCour.show(10)

+----------------+----+-----+
|        CallType| Box|count|
+----------------+----+-----+
|Medical Incident|2251| 1544|
|Medical Incident|1453| 1491|
|Medical Incident|1461| 1205|
|Medical Incident|5236| 1015|
|Medical Incident|1545|  933|
|Medical Incident|2252|  913|
|Medical Incident|2248|  873|
|Medical Incident|1365|  822|
|Medical Incident|1546|  818|
|Medical Incident|2242|  816|
+----------------+----+-----+
only showing top 10 rows



appelCour: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, Box: string ... 1 more field]


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [21]:
//Reponse 5-b
val fewFireDF = fireDF.select("Neighborhood", "ZipCode").
                where($"ZipCode" === 94102 || $"ZipCode" === 94103 && $"City" === "San Francisco").
                distinct()
fewFireDF.show()

+--------------------+-------+
|        Neighborhood|ZipCode|
+--------------------+-------+
|        Potrero Hill|  94103|
|    Western Addition|  94102|
|          Tenderloin|  94102|
|            Nob Hill|  94102|
| Castro/Upper Market|  94103|
|     South of Market|  94102|
|     South of Market|  94103|
|        Hayes Valley|  94103|
|Financial Distric...|  94102|
|         Mission Bay|  94103|
|          Tenderloin|  94103|
|Financial Distric...|  94103|
|        Hayes Valley|  94102|
|             Mission|  94103|
+--------------------+-------+



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, ZipCode: int]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [22]:
//Reponse 6
fireDF.select("Delay").describe().show()

+-------+-----------------+
|summary|            Delay|
+-------+-----------------+
|  count|           175296|
|   mean|3.892364154521585|
| stddev|9.378286226254206|
|    min|      0.016666668|
|    max|          1844.55|
+-------+-----------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

Nous allons d'abord afficher les années distincts

In [23]:
val anneDist = fireTSDF.select(year($"IncidentDate")).distinct
anneDist.show()

+------------------+
|year(IncidentDate)|
+------------------+
|              2003|
|              2007|
|              2018|
|              2015|
|              2006|
|              2013|
|              2014|
|              2004|
|              2012|
|              2009|
|              2016|
|              2001|
|              2005|
|              2000|
|              2010|
|              2011|
|              2008|
|              2017|
|              2002|
+------------------+



anneDist: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [year(IncidentDate): int]


Et maintenant nous allons calculer le nombre d'années distincts avec la fonction count()

In [25]:
val nbrAnneDistinct = anneDist.count()
print(s"le nombre d'années distinctes est: $nbrAnneDistinct")

nbrAnneDistinct: Long = 19


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

Nous allons d'abord recurer les enregistrement 2018 et leur semaine

In [28]:
//Reponse 7-b
val enreg2018AndWeek = fireTSDF.select(year($"IncidentDate").as("date"),
                weekofyear(col("IncidentDate")).as("weekofyear")).
                filter($"date"==="2018")
enreg2018AndWeek.show()

enreg2018AndWeek: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, weekofyear: int]


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [29]:
//Reponse 8
val enreg2018 = fireTSDF.select(year($"IncidentDate").
                as("date"),$"ResponseDelayedinMins",$"Neighborhood").
                filter($"date"==="2018")
enreg2018.show(10)

enreg2018: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, ResponseDelayedinMins: float ... 1 more field]


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [30]:
//Reponse 9
fewFireDF.write.parquet("fewFireDF.parquet")

java.lang.RuntimeException:  java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\Hadoop\bin\bin -see https://wiki.apache.org/hadoop/WindowsProblems

### Question 10
**Comment relire les données stockée en format Parquet?**

In [31]:
//Reponse 10
spark.read.parquet("fewFireDF.parquet").show()

java.lang.UnsatisfiedLinkError:  org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z

## FIN